# Custom Agent

I had limited success with the built-in functions, so I'll test a custom flow, really a combination of tweaked langchain functions.

I think part of this could be more specialized schema and table extraction, but for now I think I'll stick with my single schema example and try to expand later.

### Imports

In [20]:
import pandas as pd
import os
from dotenv import load_dotenv

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFaceHub, SQLDatabase

### Setup query
We'll use this to input into the model

In [21]:
question = "How many heads of the departments are older than 56?"

### Load Target Tables from Vectordb

Load in the top 3 results and save just the unique schemas to a set.

In [23]:
#setup embeddings using HuggingFace and the directory location
embeddings  = HuggingFaceEmbeddings()
persist_dir = '../data/processed/chromadb/schema-table-split'

# load from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

#run similarity search
top_results = vectordb.similarity_search(question, k=3) #just return the top for now

In [28]:
top_targets[0].metadata['schema']

'department_management'

#### Setup Target Schemas

In [34]:
target_schemas = set()
for doc in top_targets:
    schema = doc.metadata['schema']
    target_schemas.add(schema)

print(target_schemas)

{'department_management', 'hr_1'}


In [37]:
list(target_schemas)[0]

'department_management'

### Tool - Query for Schema Information
Use the langchain SQLDatabase class to get all the information about the schema. I think I also want to use the most likely columns returned to sort and prioritize how the tables are fed into the llm.

#### Point to Database

In [38]:
db_filepath = '../data/processed/db/'
db_filename = list(target_schemas)[0] + '.sqlite'

In [41]:
#point to database
base_dir = os.path.dirname(os.path.abspath(db_filepath+db_filename)) #get the full path within the device
db_path = os.path.join(base_dir, db_filename) #combine with filename to get db_path
db = SQLDatabase.from_uri("sqlite:///" + db_path) #connect via the lanchain method

#### Set table sorting and then pull tables names using langchain

In [64]:
#pull the table names from the vector query so we can have them in order of similarity to the question.
table_sorting = set()
for doc in top_targets:
    schema = doc.metadata['schema']
    if schema == 'department_management':
        table_name = doc.metadata['table']
        table_sorting.add(table_name)
table_sorting = list(table_sorting)
print(table_sorting)

['head', 'department']


In [74]:
#get tables names
table_names = db.get_usable_table_names()

#sort by the results of our vector query to load the most likely table in first. Not sure if this will make a difference, but can't imagine it would hurt.
priority_tables = sorted(table_names, key=lambda x: (table_sorting.index(x) if x in table_sorting else float('inf'), table_names.index(x)))

priority_tables

['head', 'department', 'management']

#### Get SQL Dialect

Using the langchain class again

In [68]:
sql_dialect = db.dialect

sql_dialect

'sqlite'

#### Get table info

You can do this for all tables in the database automatically with the function, but I want to continue trying to order these by likelihood of being related to the question. So I'll do this in a loop.

This gives the full create statement and 3 sample rows, which I think will be a great start. And really gets us to where the Chain does for adding to the prompt template.

In [80]:
table_info = []
for table in priority_tables:
    info = db.get_table_info(table_names=[table])
    table_info.append(info)

table_info

['head']
['department']
['management']


['\nCREATE TABLE head (\n\t"head_ID" INTEGER, \n\tname TEXT, \n\tborn_state TEXT, \n\tage REAL, \n\tPRIMARY KEY ("head_ID")\n)\n\n/*\n3 rows from head table:\nhead_ID\tname\tborn_state\tage\n1\tTiger Woods\tAlabama\t67.0\n2\tSergio García\tCalifornia\t68.0\n3\tK. J. Choi\tAlabama\t69.0\n*/',
 '\nCREATE TABLE department (\n\t"Department_ID" INTEGER, \n\t"Name" TEXT, \n\t"Creation" TEXT, \n\t"Ranking" INTEGER, \n\t"Budget_in_Billions" REAL, \n\t"Num_Employees" REAL, \n\tPRIMARY KEY ("Department_ID")\n)\n\n/*\n3 rows from department table:\nDepartment_ID\tName\tCreation\tRanking\tBudget_in_Billions\tNum_Employees\n1\tState\t1789\t1\t9.96\t30266.0\n2\tTreasury\t1789\t2\t11.1\t115897.0\n3\tDefense\t1947\t3\t439.3\t3000000.0\n*/',
 '\nCREATE TABLE management (\n\t"department_ID" INTEGER, \n\t"head_ID" INTEGER, \n\ttemporary_acting TEXT, \n\tPRIMARY KEY ("department_ID", "head_ID"), \n\tFOREIGN KEY("head_ID") REFERENCES head ("head_ID"), \n\tFOREIGN KEY("department_ID") REFERENCES department 

In [5]:
BASE_DIR = os.path.dirname(os.path.abspath('../data/processed/db/'+top_schema+'.sqlite'))
db_path = os.path.join(BASE_DIR, "department_management.sqlite")

db = SQLDatabase.from_uri("sqlite:///" + db_path)

### Setup LLM
I'll continue to use the falcon-7b.

In [15]:
#get api key
load_dotenv()
hf_api_token = os.getenv('hf_token')

#add path to HF repo
repo_id = 'tiiuae/falcon-7b-instruct'

#establish llm model
llm = HuggingFaceHub(repo_id=repo_id, huggingfacehub_api_token=hf_api_token, model_kwargs={"temperature": 0.5, "max_length": 8000})

## Create Agent

https://python.langchain.com/docs/modules/agents/

Two types ->
- Action: at each step, decide on the next action using the outputs of the previous steps
- Plan & Execute: decide on the full sequence up front, then execute them all without updating the plan

Plan & Execute is better for larger problems because it maintains focus. Often it's best to combine and let P&E agents use action agents to execute plans.

Tools -> actions an agent can take
Toolkit -> wrapper around collections of tools for specifc use cases. Will need this for sql, tool to inspect tables and tool to run queries.

#### Action Agents

Wrapped in agent executors - call the agent, get back an action and action input, call the referenced tool, take the output of it and return it back to the agent.

Typically has a prompt template, language model, and output parser

#### Plan-and-Execute Agents

Typically have the language model be the planner and the executor be an action agent.

#### Built-In Agent Observations

It appears the built-in model using an action agent - I'll try to tweak it. Maybe even by splitting out the steps into agents and trying the combo type with a larger plan & execute operating triggering the action agents.

### Agent Imports

In [7]:
from langchain.agents import initialize_agent, AgentType, load_tools, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.prompts import PromptTemplate

### Setup Toolkit

I'll use the standard langchain one.

In [8]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

### Create Prompt Template - Create SQL Query

In [16]:
create_prompt = PromptTemplate(
    input_variables={"tool_names"},
    template="""
        Use the following format:
        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        Thought: I now know the final answer
        Final Answer: SQL query ONLY, always include columns "Indexes", "Index_Name" along with your final answer. 
        If column name is a reserve word add quotes around it
        """
)

In [17]:
agent_exec = create_sql_agent(llm=llm,
                                toolkit=toolkit,
                                verbose=True,
                                format_instructions=create_prompt,
                                top_k=5)

In [18]:
agent_exec("How many heads of the departments are older than 56?")



> Entering new  chain...
Action: sql_db_query
Action Input: 
Observation: 
Thought: I now know the final answer
Final Answer: 

SELECT COUNT(*) FROM department WHERE age

> Finished chain.


{'input': 'How many heads of the departments are older than 56?',
 'output': 'SELECT COUNT(*) FROM department WHERE age'}

### Create Prompt Template - Check the SQL Query

From here: https://canvasapp.com/blog/text-to-sql-in-production

In [25]:
prompt = PromptTemplate(
    input_variables=["question", "sql_dialect", "current_day", "sql"],
    template="""
        You are a helpful AI that verifies that a SQL query runs correctly. If the query does not run successfully, you iteratively update the query based on the error message.
        You follow the following procedure:
            1. Run the input SQL query using the QueryTool
            2. If this runs successfully, return immediately.
            3. If the SQL query fails, debug the query:
                3a. consider the error message
                3b. update the SOL query
                3c. try re-running
            4. Repeat step 3 until you have a valid result. Finish and exit with the updated SQL query.
        You are writing SQL for the {sql_dialect} dialect.
        The current day is {current_day} in case the user references the date
        The user's original question: {question}
        The SQL: {sql}
        Begin!
    """
)

In [27]:
formatted_prompt = prompt.format(
    question=question,
    sql=sql, #feed in an sql from a prior step?
    sql_dialect=sql_dialect,
)

NameError: name 'sql' is not defined